In [1]:
import numpy as np
import pandas as pd
import torch 
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

In [2]:
import preprocessingWithMissingvalues

carDf= preprocessingWithMissingvalues.preprocess('../ann/cardata.csv')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15420 entries, 0 to 15419
Data columns (total 33 columns):
Month                   15420 non-null object
WeekOfMonth             15420 non-null int64
DayOfWeek               15420 non-null object
Make                    15420 non-null object
AccidentArea            15420 non-null object
DayOfWeekClaimed        15420 non-null object
MonthClaimed            15420 non-null object
WeekOfMonthClaimed      15420 non-null int64
Sex                     15420 non-null object
MaritalStatus           15420 non-null object
Age                     15420 non-null int64
Fault                   15420 non-null object
PolicyType              15420 non-null object
VehicleCategory         15420 non-null object
VehiclePrice            15420 non-null object
FraudFound              15420 non-null object
PolicyNumber            15420 non-null int64
RepNumber               15420 non-null int64
Deductible              15420 non-null int64
DriverRating           

/usr/local/anaconda35/lib/python3.6/site-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


In [3]:
#get the lable from the dataset
carDf.head()
carLable= carDf['Lable']
carDf.drop(['Lable'],inplace=True,axis=1) #drop the lable;


In [4]:
#apply the PCA
from sklearn.decomposition import PCA

pca = PCA(.95)
pca_result = pca.fit_transform(carDf)
#pca_df = pd.DataFrame(columns = ['pca1','pca2','pca3','pca4'])

In [5]:
print(type(pca_result))

<class 'numpy.ndarray'>


In [6]:
pca_result= pd.DataFrame(pca_result)
pca_result.head()

,0,1,2,3,4,5,6,7,8,9,...,24,25,26,27,28,29,30,31,32,33
0,0.922849,0.132363,-1.117156,0.898516,-0.159263,0.163660,-0.419927,0.994513,0.082305,0.968195,...,0.132322,-0.257047,0.438373,-0.036248,-0.099411,0.067951,0.631754,-0.126064,0.646724,-0.262620
1,1.295603,0.231041,-1.291208,0.607421,-0.041502,0.569090,-0.049636,-0.088628,-0.328559,-0.167960,...,-0.201041,-0.431792,0.022621,-0.226266,1.339788,0.069940,-0.220712,-0.435173,0.553638,-0.376443
2,-0.622890,-0.115813,-0.577577,0.881442,-0.310303,0.305865,0.197651,0.111288,0.644124,-0.580193,...,0.020538,0.149915,0.286175,-0.111701,-0.049428,0.111080,-0.229686,-0.317591,0.510877,-0.409610
3,-0.729731,0.539017,0.196202,-0.890831,0.530871,-0.791703,-0.204858,-0.015203,1.105974,-0.568063,...,-0.439378,0.227609,0.325784,0.106025,1.364357,-0.044349,-0.036467,-0.235350,-0.043242,0.085156
4,1.360981,1.062972,-0.722035,0.906465,0.621309,-0.119266,0.176604,0.514143,0.243289,1.018288,...,0.509380,0.347518,-0.164710,-0.454598,-0.046579,0.238470,-0.306709,-0.401991,0.343102,-0.327954


In [7]:
carDateNormalized= pca_result

In [8]:
#Divide the data into fraudulent and non-fraudulent data
columnlable= list(carDateNormalized.columns.values)

#create the dataframe for fraudulent and non-fraudulent data
nonFraudulent= pd.DataFrame(columns=columnlable)
nonFraudulentLable= pd.DataFrame(columns=['lable'])

fraudulent= pd.DataFrame(columns=columnlable)
fraudulentLable= pd.DataFrame(columns=['lable'])


j= 0
k= 0
for i in range(carDateNormalized.shape[0]):
    if(carLable[i]==0):
        nonFraudulent.loc[j]= carDateNormalized.loc[i]
        nonFraudulentLable.loc[j]= 0.0
        j += 1
    else:
        fraudulent.loc[k]= carDateNormalized.loc[i]
        fraudulentLable.loc[i]= 1.0
        k += 1

In [9]:
print(nonFraudulent.shape)
print(fraudulent.shape)
nonFraudulent.head()

(14496, 34)
(923, 34)


,0,1,2,3,4,5,6,7,8,9,...,24,25,26,27,28,29,30,31,32,33
0,0.922849,0.132363,-1.117156,0.898516,-0.159263,0.163660,-0.419927,0.994513,0.082305,0.968195,...,0.132322,-0.257047,0.438373,-0.036248,-0.099411,0.067951,0.631754,-0.126064,0.646724,-0.262620
1,1.295603,0.231041,-1.291208,0.607421,-0.041502,0.569090,-0.049636,-0.088628,-0.328559,-0.167960,...,-0.201041,-0.431792,0.022621,-0.226266,1.339788,0.069940,-0.220712,-0.435173,0.553638,-0.376443
2,-0.622890,-0.115813,-0.577577,0.881442,-0.310303,0.305865,0.197651,0.111288,0.644124,-0.580193,...,0.020538,0.149915,0.286175,-0.111701,-0.049428,0.111080,-0.229686,-0.317591,0.510877,-0.409610
3,-0.729731,0.539017,0.196202,-0.890831,0.530871,-0.791703,-0.204858,-0.015203,1.105974,-0.568063,...,-0.439378,0.227609,0.325784,0.106025,1.364357,-0.044349,-0.036467,-0.235350,-0.043242,0.085156
4,1.360981,1.062972,-0.722035,0.906465,0.621309,-0.119266,0.176604,0.514143,0.243289,1.018288,...,0.509380,0.347518,-0.164710,-0.454598,-0.046579,0.238470,-0.306709,-0.401991,0.343102,-0.327954


In [10]:
#convert to dataframe
nonFraudulentLable= pd.DataFrame(nonFraudulentLable)
fraudulentLable= pd.DataFrame(fraudulentLable)


In [11]:
nonFraudulent.head()
fraudulentLable.head()

,lable
28,1.0
52,1.0
53,1.0
94,1.0
96,1.0


In [12]:
#divide the nonFraudulent into test and train
nonFraudX_train,nonFraudX_test,nonFraudY_train,nonFraudY_test = train_test_split(nonFraudulent,nonFraudulentLable,random_state=3,test_size=0.25)
print('nonFraudxtrain:',type(nonFraudX_train))
print(nonFraudX_test.shape)

#divide the fraudulent into test and train
fraudX_train,fraudX_test,fraudY_train,fraudY_test = train_test_split(fraudulent, fraudulentLable,random_state=3,test_size=0.25)
print('fraudxtrain:',type(fraudX_train))
print(fraudX_test.shape)

nonFraudxtrain: <class 'pandas.core.frame.DataFrame'>
(3624, 34)
fraudxtrain: <class 'pandas.core.frame.DataFrame'>
(231, 34)


In [13]:
#convert into numpy
nonFraudX_train= nonFraudX_train.values
nonFraudX_test= nonFraudX_test.values
nonFraudY_train= nonFraudY_train.values
nonFraudY_test= nonFraudY_test.values

fraudX_train= fraudX_train.values
fraudX_test= fraudX_test.values
fraudY_train= fraudY_train.values
fraudY_test= fraudY_test.values


In [14]:
#create the class Dataset which returns the data and labels
from torch.utils.data.dataset import Dataset, TensorDataset
class myDataset(Dataset):
    
    def __init__(self,features,lables,transform= None):
        dataTensor= []
        lableTensor= []
        dataSize= features.shape[0]
        
        for data in range(dataSize):
            feature= features[data,:]
            #feature= torch.from_numpy(feature).float()
            feature= torch.Tensor(feature)
            dataTensor.append(feature)
            
            lable= np.asanyarray(lables[data])
            lable= torch.from_numpy(lable).float()
           
            #lable= torch.Tensor(lable)
            #print("lable:",lable)
            #assert(False)
           
            lableTensor.append(lable)
        
        #put everything in features and lables
        self.features= dataTensor
        self.lables= lableTensor
    
    
    def __len__(self):
        return len(self.features)
    
    def __getitem__(self,index):
        
        feature= self.features[index]
        lable= self.lables[index]
        #print("get_item feature:",feature)
        #print("get_item lable:",lable)
        return feature,lable

In [15]:
#make the  nonfraudulent dataset for train and test loader

myNonFraudulentTrainDataset= myDataset(nonFraudX_train,nonFraudY_train)
myNonFraudulentTestDataset= myDataset(nonFraudX_test,nonFraudY_test)

#make the fraudulent dataset for train and test loader
myFraudulentTrainDataset= myDataset(fraudX_train,fraudY_train)
myFraudulentTestDataset= myDataset(fraudX_test,fraudY_test)

In [16]:
#make the trainloader and test loader for nonfraudulent dataset.
nonFraudulentTrainLoader= torch.utils.data.DataLoader(myNonFraudulentTrainDataset,batch_size=1,shuffle=True,num_workers=0)
nonFraudulentTestLoader= torch.utils.data.DataLoader(myNonFraudulentTestDataset,batch_size=1,shuffle=True,num_workers=0)

fraudulentTrainLoader= torch.utils.data.DataLoader(myFraudulentTrainDataset,batch_size=1,shuffle=True,num_workers=0)
fraudulentTestLoader= torch.utils.data.DataLoader(myFraudulentTestDataset,batch_size=1,shuffle=True,num_workers=0)



In [17]:
#Network architecture for the base autoencoders
class autoencoder1(nn.Module):
    def __init__(self):
        super(autoencoder1,self).__init__()
        self.encoder1= nn.Sequential(
            nn.Linear(34,70),
            nn.LeakyReLU(True),
            nn.Linear(70,60),
            nn.LeakyReLU(True),
            nn.Linear(60,45),
            nn.LeakyReLU(True),
            nn.Linear(45,20),nn.LeakyReLU(True),nn.Linear(20,10))
        
        self.decoder1= nn.Sequential(
            nn.Linear(10,20),
            nn.LeakyReLU(True),
            nn.Linear(20,45),
            nn.LeakyReLU(True),
            nn.Linear(45,60),
            nn.LeakyReLU(True),
            nn.Linear(60,70),nn.LeakyReLU(True),nn.Linear(70,34),nn.ReLU())
    
    def forward(self,x):
        x= self.encoder1(x)
        x= self.decoder1(x)
        return x;

In [18]:
model1= autoencoder1()
criterion1= nn.MSELoss()
optimizer1= torch.optim.SGD(model1.parameters(), lr=0.001,momentum=0.9)

In [19]:
#train the model
for epoch in range(50):
    running_loss= 0.0
    for i, (feature1,lable1) in enumerate(nonFraudulentTrainLoader):
        
        #gets the inputs
        inputs1= torch.tensor(feature1)
        lables1= torch.tensor(lable1)
        lables1= lables1.type(torch.LongTensor)
        
        # =====================forward====================
        output1 = model1(inputs1)
        loss1 = criterion1(output1,inputs1)
        
         # ===================backward====================
        optimizer1.zero_grad()
        loss1.backward()
        optimizer1.step()
        
        # =======print the statistics
        running_loss += loss1.item()
        
        #print("i: ",i)
    #if i%100 == 0:              #print every 2000 mini-batches
    
    print('[%d] loss: %.3f' %
              (epoch + 1,  running_loss /nonFraudX_train.shape[0]))
      #running_loss = 0.0

        

[1] loss: 0.196
[2] loss: 0.196
[3] loss: 0.193
[4] loss: 0.190
[5] loss: 0.190
[6] loss: 0.190
[7] loss: 0.190
[8] loss: 0.190
[9] loss: 0.189
[10] loss: 0.188
[11] loss: 0.185
[12] loss: 0.180
[13] loss: 0.176
[14] loss: 0.174
[15] loss: 0.174
[16] loss: 0.173
[17] loss: 0.173
[18] loss: 0.172
[19] loss: 0.172
[20] loss: 0.172
[21] loss: 0.172
[22] loss: 0.172
[23] loss: 0.170
[24] loss: 0.168
[25] loss: 0.167
[26] loss: 0.167
[27] loss: 0.167
[28] loss: 0.165
[29] loss: 0.165
[30] loss: 0.164
[31] loss: 0.164
[32] loss: 0.164
[33] loss: 0.164
[34] loss: 0.164
[35] loss: 0.163
[36] loss: 0.162
[37] loss: 0.161
[38] loss: 0.160
[39] loss: 0.159
[40] loss: 0.158
[41] loss: 0.156
[42] loss: 0.155
[43] loss: 0.155
[44] loss: 0.154
[45] loss: 0.154
[46] loss: 0.153
[47] loss: 0.153
[48] loss: 0.152
[49] loss: 0.152
[50] loss: 0.151


In [20]:
 #test the model
def modelTest1(Loader):
    lossList1= []
    trueLable1= []
    for i,(feature1,lable1) in enumerate(Loader):
        inputs1= torch.tensor(feature1)
        output1= model1(inputs1)
        loss1= criterion1(output1,inputs1)
        trueLable1.append(lable1)
        lossList1.append(loss1.item())
        lossValue1= loss1.item()
        f= open("./modle1Loss.txt",'a')
        f.write(str(lossValue1) + '\n')
        '''
        if(i!=10):
            print("input:",inputs1)
            print("output:",output1)
            print("loss1:",loss1)
            print("lable:",lable1)
        else:
            assert(False)
        '''
    f.close()
    return lossList1,trueLable1
        

In [21]:
#test the model
lossList1,trueLable1= modelTest1(nonFraudulentTestLoader)
len(lossList1)
lossList1[1]

0.16235050559043884

In [22]:
#Network architecture for the base autoencoders
class autoencoder2(nn.Module):
    def __init__(self):
        super(autoencoder2,self).__init__()
        self.encoder2= nn.Sequential(
            nn.Linear(34,87),
            nn.LeakyReLU(True),
            nn.Linear(87,77),
            nn.LeakyReLU(True),
            nn.Linear(77,55),
            nn.LeakyReLU(True),
            nn.Linear(55,45),
            nn.LeakyReLU(True),
            nn.Linear(45,25),nn.LeakyReLU(True),nn.Linear(25,10))
       
        self.decoder2= nn.Sequential(
            nn.Linear(10,25),
            nn.LeakyReLU(True),
            nn.Linear(25,45),
            nn.LeakyReLU(True),
            nn.Linear(45,55),
            nn.LeakyReLU(True),
            nn.Linear(55,77),
            nn.LeakyReLU(True),
            nn.Linear(77,87),nn.LeakyReLU(True),nn.Linear(87,34),nn.ReLU())
    
    def forward(self,x):
        x= self.encoder2(x)
        x= self.decoder2(x)
        return x;

In [23]:
model2= autoencoder2()
criterion2= nn.MSELoss()
optimizer2= torch.optim.SGD(model2.parameters(), lr=0.001,momentum=0.9)

In [24]:
#train the model2
for epoch in range(300):
    running_loss2= 0.0
    for i, (feature2,lable2) in enumerate(fraudulentTestLoader):
        
        #gets the inputs
        inputs2= torch.tensor(feature2)
        lables2= torch.tensor(lable2)
        lables2= lables2.type(torch.LongTensor)
       
        # =====================forward====================
        output2= model2(inputs2)
        loss2= criterion2(output2,inputs2)
        
        # ===================backward====================
        optimizer2.zero_grad()
        loss2.backward()
        optimizer2.step()
        
        # =======print the statistics
        running_loss2 += loss2.item()
        
        #print("i: ",i)
    #if i%100 == 0:              #print every 2000 mini-batches
    
    print('[%d] loss: %.3f' %
              (epoch + 1,  running_loss2 / fraudX_train.shape[0]))
      #running_loss = 0.0

[1] loss: 0.062
[2] loss: 0.062
[3] loss: 0.062
[4] loss: 0.062
[5] loss: 0.062
[6] loss: 0.062
[7] loss: 0.062
[8] loss: 0.062
[9] loss: 0.062
[10] loss: 0.062
[11] loss: 0.062
[12] loss: 0.062
[13] loss: 0.062
[14] loss: 0.062
[15] loss: 0.062
[16] loss: 0.062
[17] loss: 0.062
[18] loss: 0.062
[19] loss: 0.062
[20] loss: 0.062
[21] loss: 0.062
[22] loss: 0.062
[23] loss: 0.062
[24] loss: 0.062
[25] loss: 0.062
[26] loss: 0.062
[27] loss: 0.062
[28] loss: 0.062
[29] loss: 0.062
[30] loss: 0.062
[31] loss: 0.062
[32] loss: 0.062
[33] loss: 0.062
[34] loss: 0.062
[35] loss: 0.062
[36] loss: 0.062
[37] loss: 0.062
[38] loss: 0.062
[39] loss: 0.062
[40] loss: 0.062
[41] loss: 0.062
[42] loss: 0.062
[43] loss: 0.062
[44] loss: 0.062
[45] loss: 0.062
[46] loss: 0.062
[47] loss: 0.062
[48] loss: 0.062
[49] loss: 0.062
[50] loss: 0.062
[51] loss: 0.062
[52] loss: 0.062
[53] loss: 0.062
[54] loss: 0.062
[55] loss: 0.062
[56] loss: 0.062
[57] loss: 0.062
[58] loss: 0.062
[59] loss: 0.062
[60] l

In [25]:
def modelTest2(Loader2):
    lossList2= []
    trueLable1= []
    for i,(feature2,lable2) in enumerate(Loader2):
        inputs2= torch.tensor(feature2)
        output2= model2(inputs2)
        loss2= criterion2(output2,inputs2)
        trueLable1.append(lable2)
        lossList2.append(loss2.item()) #put the loss
        '''        
        if(i!=10):
            print("input:",inputs2)
            print("output:",output2)
            print("loss1:",loss2)
            print("lable:",lable2)
        else:
            assert(False)
        '''
        lossValue2= str(loss2.item())
        f= open("./modle2Loss.txt",'a')
        f.write(lossValue2+'\n')
    f.close()
        
    return lossList2,trueLable1
        

In [26]:
lossList2,trueLable1= modelTest2(nonFraudulentTestLoader)


In [27]:
predictedLable1=[]
for k in range(len(lossList1)):
    if(lossList1[k]<lossList2[k]):
        predictedLable1.append(0)
    else:
        predictedLable1.append(1)

In [28]:
#test model1 with another type of data
lossList3,trueLable2= modelTest1(fraudulentTestLoader)


In [29]:
#test model2 with another type of data
lossList4,trueLable2= modelTest2(fraudulentTestLoader)


In [30]:
predictedLable2=[]
for k in range(len(lossList3)):
    if(lossList3[k]<lossList4[k]):
        predictedLable2.append(0)
    else:
        predictedLable2.append(1)

In [31]:
totalLength1= len(trueLable1)
TP=FP=FN=TN= 0
for i in range(totalLength1):
    if(int(trueLable1[i])==1 and predictedLable1[i]==1):
        TP += 1
    elif(int(trueLable1[i])==1 and predictedLable1[i]==0):
        FN += 1
    elif(int(trueLable1[i])==0 and predictedLable1[i]==0):
        TN += 1
    elif(int(trueLable1[i])==0 and predictedLable1[i]==1):
        FP += 1

totalLength2= len(trueLable2)
for i in range(totalLength2):
    if(int(trueLable2[i])==1 and predictedLable2[i]==1):
        TP += 1
    elif(int(trueLable2[i])==1 and predictedLable2[i]==0):
        FN += 1
    elif(int(trueLable2[i])==0 and predictedLable2[i]==0):
        TN += 1
    elif(int(trueLable2[i])==0 and predictedLable2[i]==1):
        FP += 1
        
print("\nResult ")
print("Accuracy:",(TP+TN)/(TP+FP+FN+TN)*100)
print("Sensitivity:",TP/(TP+FN)*100)
print("Specificity:",TN/(TN+FP)*100)
print("\n")
print("\nTP:",TP)
print("\nFN:",FN)
print("\nFP:",FP)
print("\nTN:",TN)



Result 
Accuracy: 78.18417639429313
Sensitivity: 29.004329004329005
Specificity: 81.31898454746137



TP: 67

FN: 164

FP: 677

TN: 2947
